<a href="https://colab.research.google.com/github/SBOSE550/Customer-Data-Processing-and-Validation-System/blob/main/auto_monthly_horeca_data_ceaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import library


In [ ]:
!pip install fuzzywuzzy
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from fuzzywuzzy import fuzz, process
import openpyxl
import os


/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Defining the function

In [ ]:
def extract_data_from_sheet(excel_file, sheet_name):
    """Extracts data from a specific sheet in an Excel file.

    Args:
        excel_file: Path to the Excel file.
        sheet_name: Name of the sheet to extract data from.

    Returns:
        A list of lists representing the data in the sheet, or None if the sheet doesn't exist.
    """
    try:
        workbook = openpyxl.load_workbook(excel_file)
        sheet = workbook[sheet_name]  # Access the sheet by name
        data = []
        for row in sheet.iter_rows():
            row_data = [cell.value for cell in row]
            data.append(row_data)
        return data
    except KeyError:
        print(f"Sheet '{sheet_name}' not found.")
        return None
    except FileNotFoundError:
        print(f"File '{excel_file}' not found.")
        return None





def reverse_geocode(lat, lon):
    geolocator = Nominatim(user_agent="my_geocoding_script")
    try:
        location = geolocator.reverse((lat, lon), timeout=10)
        if location:
            address = location.raw['address']
            city = address.get('city') or address.get('town') or address.get('village') or address.get('hamlet')
            state = address.get('state')
            postcode = address.get('postcode')  # Extract postcode
            return city, state, postcode

        else:
            return None, None
    except (GeocoderTimedOut, AttributeError):
        return None, None

# Preprocessing function
def preprocess_column(df, column1,column2):
    df[column1] = df[column1].str.strip().str.lower()
    df[column2] = df[column2].str.strip().str.lower()
    return df

def classify_existing_customer(existing_customers_df, master_df):
  mail=[]
  for index, row in existing_customers_df.iterrows():
    customer_name = row["Customer Name"]
    FPR = row["Name"]

    # Filter master list by FPR (if applicable)
    if FPR:
      FPR_customers = master_df[master_df["FPR"] == FPR]
    else:
      FPR_customers = master_df.copy()  # Consider all customers if no FPR filter

    # Check if there are no matching FPR/customers
    if FPR_customers.empty:
      print(f"No customers found for FPR: {FPR}. Skipping fuzzy matching for '{customer_name}'.")
      continue

    # Perform fuzzy matching
    matches = FPR_customers["Customer Name"].apply(lambda x: fuzz.ratio(x.lower(), customer_name.lower()))
    best_match_idx = matches.idxmax()
    best_match_score = matches.max()

    if best_match_score > 90:
      existing_customers_df.at[index, "Customer Name"] = master_df.loc[best_match_idx, "Customer Name"]
      print(f"Auto-corrected '{customer_name}' to '{master_df.loc[best_match_idx, 'Customer Name']}' under {FPR} (Score: {best_match_score})")

    elif 55 <= best_match_score <= 90:  # Adjust threshold for suggestions
      suggestion = master_df.loc[best_match_idx, "Customer Name"]
      print(f"Suggested match for '{customer_name}': '{suggestion}' Under {FPR} (Score: {best_match_score})")
      user_input = input("Replace? (yes/no): ").strip().lower()
      if user_input == 'yes':
        existing_customers_df.at[index, "Customer Name"] = suggestion
        print(f"Replaced '{customer_name}' with '{suggestion}' under {FPR}")
      else:
        print(f"Not maching with existing data '{customer_name}' under {FPR}")
        mail.append(row)

    else:
      print(f"No match found for '{customer_name}' under {FPR} (Score: {best_match_score})")
      mail.append(row)

  mail_df = pd.DataFrame(mail, columns=existing_customers_df.columns)

  print("Cleaning complet for existing customers")
  return existing_customers_df,mail_df

# Function to classify new customers based on fuzzy matching
def classify_new_customers(master_df, new_customers):
    genuine_new = []
    existing_customers = []

    for index, row in new_customers.iterrows():
        new_customer_name = row['Customer Name']
        FPR = row['Name']

        # Filter master list by salesperson (if applicable)
        if FPR:
          master_names = master_df[master_df["FPR"] == FPR]
        else:
          master_names = master_df.copy()  # Consider all customers if no salesperson filter

        # Check if there are no matching salespeople/customers
        if master_names.empty:
          print(f"No customers found for FPR: {FPR}. Skipping fuzzy matching for '{new_customer_name}'.")
          genuine_new.append(row)  # Add to genuine_new if no match found
          continue


        # Perform fuzzy matching against master dataset
       # Handle cases where extractOne might return a single element or None
        result = process.extractOne(new_customer_name, master_names['Customer Name'].tolist()) # Extract from 'Customer Name' column
        if result:
            match, score = result
        else:
            match, score = None, 0  # Default values if no match

        if score >= 90:
            # Direct match; replace with Master dataset's name
            row['Customer Name'] = match
            print(f"Potential Match Found:\nNew Customer: {new_customer_name}\nMaster Dataset Match: {match} under {FPR} (Score: {score})")
            existing_customers.append(row)
        elif 55 <= score < 90:
            # Prompt for manual input
            print(f"Potential Match Found:\nNew Customer: {new_customer_name}\nMaster Dataset Match: {match} under {FPR} (Score: {score})")
            user_input = input("Is this a match? (yes/no): ").strip().lower()

            if user_input == "yes":
                row['Customer Name'] = match
                existing_customers.append(row)
            else:
                genuine_new.append(row)
        else:
            # Genuine new customer
            genuine_new.append(row)

    # Convert lists back to DataFrames
    genuine_new_df = pd.DataFrame(genuine_new, columns=new_customers.columns)
    existing_customers_df = pd.DataFrame(existing_customers, columns=new_customers.columns)
    genuine_new_df['Business Status'] = 'Inactive'
    genuine_new_df['Sub Status'] = 'New'


    return genuine_new_df, existing_customers_df




# Load the Data and calling the Functions

In [ ]:



# Example usage
excel_file = "master workbook.xlsx"
sheet_name = "master data"
data = extract_data_from_sheet(excel_file, sheet_name)
master_df=pd.DataFrame(data[1:],columns=data[0])
new_customer_df=pd.read_html("HORECA Customer Visit Data(New Customer)_form_2025-01-21_to_2025-01-28.xls")[0]
existing_customers_df = pd.read_html("HORECA Customer Visit Data(Exiting Customer)_form_2025-01-21_to_2025-01-28.xls")[0]


# Preprocess customer names and FPR fields
master_df = preprocess_column(master_df, 'Customer Name','FPR')
new_customer_df = preprocess_column(new_customer_df, 'Customer Name','Name')
existing_customers_df = preprocess_column(existing_customers_df, 'Customer Name','Name')



# Example usage (assuming your CSV files are in the same directory)
clean_existing_customers,mail_df=classify_existing_customer(existing_customers_df,master_df )

# Get lists of master customer names and process new customers

genuine_new_df, new_existing_customers_df = classify_new_customers(master_df, new_customer_df)
genuine_new_df["Customer_type"]="new"
new_existing_customers_df["Customer_type"]="existing"
clean_existing_customers["Customer_type"]="existing"
mail_df["Customer_type"]="new"

Visit_df=pd.concat([genuine_new_df,clean_existing_customers,new_existing_customers_df,mail_df],axis=0)
# Convert 'For date' column to datetime format
Visit_df['For date'] = pd.to_datetime(Visit_df['For date'])

# Extract the date part and create a new 'Date' column
Visit_df['Date'] = Visit_df['For date'].dt.date

# Drop the 'For date' column
Visit_df.drop(columns=['For date'], inplace=True)

# Sort by 'Date'
Visit_df.sort_values(by="Date", inplace=True)
# Drop duplicate rows based on a subset of columns(it will only include one entry if the user input same cusotmer in the same da)
Visit_df.drop_duplicates(subset=['Name', 'Date', 'Customer Name'], inplace=True)




Auto-corrected 'palki foods' to 'palki foods' under mukesh gupta (Score: 100)
Suggested match for 'rituraj foods': 'rituraj foods india pvt ltd' Under mukesh gupta (Score: 65)
Replace? (yes/no): yes
Replaced 'rituraj foods' with 'rituraj foods india pvt ltd' under mukesh gupta
Auto-corrected 'thaagam foundation' to 'thangam foundation' under mukesh gupta (Score: 94)
Auto-corrected 'sampoorna marketing' to 'sampoorn marketing' under mukesh gupta (Score: 97)
No match found for 'bagrodia' under ankan chatterjee (Score: 47)
Suggested match for 'vijay traders': 'rahul traders' Under mukesh gupta (Score: 69)
Replace? (yes/no): no
Not maching with existing data 'vijay traders' under mukesh gupta
Auto-corrected 'mapple agro' to 'mapple agro' under mukesh gupta (Score: 100)
Auto-corrected 'viraat agency' to 'viraat agency' under sabyasachi guin (Score: 100)
No match found for 'virat agency' under ankan chatterjee (Score: 42)
Suggested match for 'pk enterprise': 's.m. enterprise' Under sabyasach

# Column modification for extracting dataset

In [ ]:
genuine_new_df.drop(columns=['State','City'],inplace=True)
genuine_new_df[['City', 'State','Pin Code']] = genuine_new_df.apply(lambda row: reverse_geocode(row['Latitude'], row['Longitude']), axis=1, result_type='expand')
genuine_new_df=genuine_new_df[['Customer_type','Customer Name', 'Segment Of Customer',
       'State', 'City', 'Location', 'Pin Code','Name',
       'Connected Person Designation', 'Connected Person Name',
       'Contact Number', 'Mail ID',
       'Customer Service Type', 'If Indirect Mentioned DB Name',
       'Mustard Oil (MT)', 'Soyabean Oil (MT)', 'Rice Bran Oil (MT)', 'Sunflower Oil (MT)', 'Plam Oil (MT)', 'Loose Oil (MT)',
       'Others Oils (MT)', 'Pure Spices (KG)',
       'Blend Spices (KG)', 'Soya Chunk (KG)', 'Others Foods (KG)']]
mail_df[['City', 'State','Pin Code']] = mail_df.apply(lambda row: reverse_geocode(row['Latitude'], row['Longitude']), axis=1, result_type='expand')
mail_df=mail_df[['Customer_type','Customer Name',
       'State', 'City', 'Pin Code','Name',
       'Customer Service Type', 'If Indirect Mentioned DB Name',
       'Mustard Oil (MT)', 'Soyabean Oil (MT)', 'Rice Bran Oil (MT)', 'Sunflower Oil (MT)', 'Plam Oil (MT)', 'Loose Oil (MT)',
       'Others Oils (MT)', 'Pure Spices (KG)',
       'Blend Spices (KG)', 'Soya Chunk (KG)', 'Others Foods (KG)']]

# Finalizing and extracting the final clean Data

In [ ]:
# Sort the dataset by Customer Name and Date in descending order to prioritize the latest entries
Visit_df.sort_values(by=['Customer Name', 'Date'], ascending=[True, False], inplace=True)

# Group data by unique customer name and pick the latest entries for specific columns
report_df = Visit_df.groupby('Customer Name').agg(
    FPR=('Name', 'first'),  # FPR (Field Person Responsible)
    Customer_type=('Customer_type', 'first'),  # Customer Type
    State=('State', 'first'),  # Latest State
    City=('City', 'first'),  # Latest City
    Connected_Person_Designation=('Connected Person Designation', 'last'),  # Latest Connected Person Designation
    Connected_Person=('Connected Person Name', 'last'),  # Latest Connected Person
    Connected_person_phone=('Contact Number', 'last'),  # Latest Connected Person Phone
    Sales_Connection_Type=('Customer Service Type', 'last'),  # Latest Sales Connection Type
    Distributor_Name=('If Indirect Mentioned DB Name', 'last'),  # Latest Distributor Name
    Segment=('Segment Of Customer', 'first'),  # Latest Segment
    Visit_Count=('Customer Name', 'count'),  # Count of visits
    Last_Visit_date=('Date', 'max'),  # Most recent visit date
    Business_Status=('Business Status', 'last'),  # Latest Business Status
    Sub_Status_Progression=('Sub Status', lambda x: ' <- '.join(x.astype(str).unique()))  # Concatenate unique status changes
).reset_index()

# Sort the report by 'Last Visit' in descending order
report_df.sort_values(by='Last_Visit_date', ascending=False, inplace=True)
report_df

,Customer Name,FPR,Customer_type,State,City,Connected_Person_Designation,Connected_Person,Connected_person_phone,Sales_Connection_Type,Distributor_Name,Segment,Visit_Count,Last_Visit_date,Business_Status,Sub_Status_Progression
0,alekos technologies,sabyasachi guin,existing,None,None,None,None,NaN,Direct,Na,None,1,2025-01-28,Inactive,Connection
8,jbmr caterer,ajay singh,new,up,noida,owner,mahesh sharma,9.810102e+09,Indirect,sampoorn,Catrer,1,2025-01-28,Inactive,New
25,viraat agency,sabyasachi guin,existing,None,None,None,None,NaN,Direct,None,None,1,2025-01-28,Active,Connection
23,tivoli gardens,mukesh gupta,new,delhi,delhi,purchase manager,Mr Dharam veer yadav,9.197172e+11,Indirect,Vijay traders,Hotel,1,2025-01-28,Inactive,New
19,shiva dhaba,ajay singh,new,up,gr noida,owner,puspendra,9.000701e+09,Indirect,sampoorn marketing,Hotel,1,2025-01-28,Inactive,New
14,pk enterprise,sabyasachi guin,new,None,None,None,None,NaN,Direct,None,None,1,2025-01-28,Inactive,Connection
10,mapple agro,mukesh gupta,existing,None,None,None,None,NaN,Direct,na,None,1,2025-01-28,Active,Billing
9,mahatma gandhi college noida,ajay singh,new,up,noida,owner,ajay,9.773766e+09,Indirect,sampoorn marketing,Cafe,1,2025-01-28,Inactive,New
26,virat agency,ankan chatterjee,new,None,None,None,None,NaN,Direct,na,None,1,2025-01-28,Active,Connection
2,ashirvad enterprises,ankan chatterjee,existing,None,None,None,None,NaN,Direct,na,None,1,2025-01-28,Active,Connection


In [ ]:
# Define the file name for the workbook
file_name = "Data_Report.xlsx"

# Use ExcelWriter to write multiple sheets
with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
    Visit_df.to_excel(writer, sheet_name='Visit Data', index=False)
    report_df.to_excel(writer, sheet_name='Report Data', index=False)
    genuine_new_df.to_excel(writer, sheet_name='Genuine New Data', index=False)
    mail_df.to_excel(writer, sheet_name='Mail Data', index=False)

print(f"Data successfully written to {file_name}")


Data successfully written to Data_Report.xlsx


In [ ]:
# Sort the dataset by Customer Name and Date in descending order to prioritize the latest entries
Visit_df.sort_values(by=['Customer Name', 'Date'], ascending=[True, False], inplace=True)
variable_df=Visit_df.groupby('Customer Name').agg(
              Sales_Connection_Type=('Customer Service Type', 'first'),  # Latest Sales Connection Type
              Distributor_Name=('If Indirect Mentioned DB Name', 'first'),  # Latest Distributor Name
              Business_Status=('Business Status', 'first'),  # Latest Business Status
          ).reset_index()
variable_df

,Customer Name,Sales_Connection_Type,Distributor_Name,Business_Status
0,alekos technologies,Direct,Na,Inactive
1,ambuj hotel & real estate,Indirect,Sampoorna marketing,Inactive
2,ashirvad enterprises,Direct,na,Active
3,bagrodia,Direct,None,Active
4,brijandan namkeen,None,na,Inactive
5,delhi zoological park canteen,Indirect,Vijay traders,Inactive
6,food town,None,GMKR,Inactive
7,jaypee resort and hotels,None,Rituraj foods,Inactive
8,jbmr caterer,Indirect,sampoorn,Inactive
9,mahatma gandhi college noida,Indirect,sampoorn marketing,Inactive


In [ ]:
Visit_df.columns

Index(['Id', 'User Id', 'username', 'Name', 'Latitude', 'Longitude',
       'Customer Name', 'Connected Person Designation',
       'Connected Person Name', 'Contact Number', 'Mail ID', 'Location',
       'City', 'State', 'Segment Of Customer', 'Customer Service Type',
       'If Indirect Mentioned DB Name', 'Customer Business Potential',
       'Mustard Oil (MT)', 'Soyabean Oil (MT)', 'Rice Bran Oil (MT)',
       'Sunflower Oil (MT)', 'Plam Oil (MT)', 'Loose Oil (MT)',
       'Others Oils (MT)', 'Status Remarks for Oils', 'Pure Spices (KG)',
       'Blend Spices (KG)', 'Soya Chunk (KG)', 'Others Foods (KG)',
       'Status Remarks for Foods', 'Upload Image for Visit', 'Business Status',
       'Sub Status', 'Customer_type', 'Date'],
      dtype='object')

In [ ]:
# Sort the dataset by Customer Name and Date in descending order to prioritize the latest entries
Visit_df.sort_values(by=['Customer Name', 'Date'], ascending=[True, False], inplace=True)

# Group data by unique customer name and pick the latest entries for specific columns
report_df = Visit_df.groupby('Customer Name').agg(
    FPR=('Name', 'first'),  # FPR (Field Person Responsible)
    Connected_Person=('Connected Person Name', 'last'),  # Latest Connected Person
    Sales_Connection_Type=('Customer Service Type', 'last'),  # Latest Sales Connection Type
    Distributor_Name=('If Indirect Mentioned DB Name', 'last'),  # Latest Distributor Name
    Segment=('Segment Of Customer', 'first'),  # Latest Segment
    Visit_Count=('Customer Name', 'count'),  # Count of visits
    Last_Visit_date=('Date', 'max'),  # Most recent visit date
    Business_Status=('Business Status', 'last'),  # Latest Business Status
    Sub_Status_Progression=('Sub Status', lambda x: ' <- '.join(x.astype(str).unique()))  # Concatenate unique status changes
).reset_index()

# Sort the report by 'Last Visit' in descending order
report_df.sort_values(by='Last_Visit_date', ascending=False, inplace=True)
report_df

# # Merge master dataset and report_df based on FPR and Customer Name
# merged_df = pd.merge(
#     report_df,
#     master_df[['Customer Name', 'FPR', 'Segment', 'Connected Person Phone']],  # Select relevant columns
#     on=['Customer Name', 'FPR'],  # Merge on these columns
#     how='left'  # Perform a left join to retain all data in report_df
# )

# # Display the merged DataFrame
# merged_df

,Customer Name,FPR,Connected_Person,Sales_Connection_Type,Distributor_Name,Segment,Visit_Count,Last_Visit_date,Business_Status,Sub_Status_Progression
0,alekos technologies,sabyasachi guin,None,Direct,Na,None,1,2025-01-28,Inactive,Connection
8,jbmr caterer,ajay singh,mahesh sharma,Indirect,sampoorn,Catrer,1,2025-01-28,Inactive,New
25,viraat agency,sabyasachi guin,None,Direct,None,None,1,2025-01-28,Active,Connection
23,tivoli gardens,mukesh gupta,Mr Dharam veer yadav,Indirect,Vijay traders,Hotel,1,2025-01-28,Inactive,New
19,shiva dhaba,ajay singh,puspendra,Indirect,sampoorn marketing,Hotel,1,2025-01-28,Inactive,New
14,pk enterprise,sabyasachi guin,None,Direct,None,None,1,2025-01-28,Inactive,Connection
10,mapple agro,mukesh gupta,None,Direct,na,None,1,2025-01-28,Active,Billing
9,mahatma gandhi college noida,ajay singh,ajay,Indirect,sampoorn marketing,Cafe,1,2025-01-28,Inactive,New
26,virat agency,ankan chatterjee,None,Direct,na,None,1,2025-01-28,Active,Connection
2,ashirvad enterprises,ankan chatterjee,None,Direct,na,None,1,2025-01-28,Active,Connection


In [ ]:
market_df = pd.read_html("Market  Visit Report_form_2025-01-01_to_2025-01-29.xls")[0]
# prompt: drop the row form market_df when Name =['Sirsha Basu,''Manoj Gond']

market_df = market_df[~market_df['Name'].isin(['Sirsha Basu', 'Manoj Gond'])]
market_df[['City', 'State','Pin Code']] = market_df.apply(lambda row: reverse_geocode(row['Latitude'], row['Longitude']), axis=1, result_type='expand')
# Get the actual column names from the DataFrame
actual_columns = market_df.columns.tolist()

# Columns to drop (correct any typos if needed)
columns_to_drop = [
    'Outlet area or Location', 'Outlet Image',
    'Worked with Merchandiser', 'Merchandiser Name', 'Emami Stock Checked',
    'Image of Emami Stocks', 'Stock Check Observation (if any)',
    'Compititor Price Check', 'Competitor Brand', 'Oil Type',
    'Competitor PTC', 'Competitor Execution Image','Pin Code'
]
market_df.rename(columns={'Outlet Name': 'Customer Name'}, inplace=True)
# Find the common columns between the desired and actual columns
valid_columns_to_drop = list(set(columns_to_drop) & set(actual_columns))

# Drop only the valid columns
market_df.drop(columns=valid_columns_to_drop, inplace=True)
new_df=pd.concat([new_customer_df,market_df])

<ipython-input-11-c605864b53f3>:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  market_df = pd.read_html("Market  Visit Report_form_2025-01-01_to_2025-01-29.xls")[0]


ValueError: No tables found

In [ ]:
new_df['Customer Name'].unique()

In [ ]:
market_df = pd.read_html("Market  Visit Report_form_2025-01-01_to_2025-01-29.xls")[0]
# prompt: drop the row form market_df when Name =['Sirsha Basu,''Manoj Gond']

market_df = market_df[~market_df['Name'].isin(['Sirsha Basu', 'Manoj Gond'])]
market_df[['City', 'State','Pin Code']] = market_df.apply(lambda row: reverse_geocode(row['Latitude'], row['Longitude']), axis=1, result_type='expand')
# Get the actual column names from the DataFrame
actual_columns = market_df.columns.tolist()

# Columns to drop (correct any typos if needed)
columns_to_drop = [
    'Outlet area or Location', 'Outlet Image',
    'Worked with Merchandiser', 'Merchandiser Name', 'Emami Stock Checked',
    'Image of Emami Stocks', 'Stock Check Observation (if any)',
    'Compititor Price Check', 'Competitor Brand', 'Oil Type',
    'Competitor PTC', 'Competitor Execution Image','Pin Code'
]
market_df.rename(columns={'Outlet Name': 'Customer Name'}, inplace=True)
# Find the common columns between the desired and actual columns
valid_columns_to_drop = list(set(columns_to_drop) & set(actual_columns))

# Drop only the valid columns
market_df.drop(columns=valid_columns_to_drop, inplace=True)
new_df=pd.concat([new_customer_df,market_df])